In [55]:
import zarr as z
import torch
import numpy as np

import sys
sys.path.insert(0, "../../src")

from juart.conopt.functional.fourier import (
    fourier_transform_adjoint,
    fourier_transform_forward,
    nonuniform_fourier_transform_adjoint,
)

store = z.open("/home/jovyan/datasets/num_phantom_128_R1")

C = torch.from_numpy(np.array(store["C"]))
k = torch.from_numpy(np.array(store["k"]))[...,None,None]
d = torch.from_numpy(np.array(store["d"]))[...,None,None]

In [56]:
AHd = nonuniform_fourier_transform_adjoint(k,d,(128,128,128))

In [57]:
AHd.shape, d.shape, k.shape

(torch.Size([8, 128, 128, 128, 1, 1]),
 torch.Size([8, 2097152, 1, 1]),
 torch.Size([3, 2097152, 1, 1]))

In [58]:
kspace_mask_worker0 = torch.randint(0, 2, (1, k.shape[1], 1, 1))
kspace_mask_worker1 = 1 - kspace_mask_worker0

In [59]:
k_scaled_masked = k * kspace_mask_worker0
AHd = nonuniform_fourier_transform_adjoint(k_scaled_masked, d, (128, 128, 128))
AHd = torch.sum(torch.conj(C[...,None,None]) * AHd, dim=0)

In [60]:
print(k_scaled_masked.shape,AHd.shape,C.shape,d.shape)

torch.Size([3, 2097152, 1, 1]) torch.Size([128, 128, 128, 1, 1]) torch.Size([8, 128, 128, 128]) torch.Size([8, 2097152, 1, 1])
